In [1]:
from langame import LangameClient
import json
import openai
import datetime
c = LangameClient("../config.yaml")

# topic classification

In [ ]:
out_file_name = f"../data/fine_tune_topic_classification_{datetime.date.today().strftime('%d_%m_%Y')}.jsonl"

with open(out_file_name, 'w') as outfile:
    for e in c._firestore_client.collection("memes").stream():
        json.dump({
            "text": e.get("content"),
            "label": e.get("topics")[0]
        }, outfile)
        outfile.write('\n')

In [ ]:
c_f = openai.File.create(
    file=open(file_name),
    purpose="classifications"
)

In [ ]:
openai.Classification.create(
    file="file-rV9rZbiiziXLfjqc4MKLLChy",
    query="123",
    search_model="ada", 
    model="curie", 
    max_examples=3
)

# quality classification

In [2]:
from langame.conversation_starters import get_existing_conversation_starters
import logging
logger = logging.getLogger("classification")
memes, index, embeddings_model = get_existing_conversation_starters(
    c._firestore_client, logger=logger, confirmed=True)

Skipping id: 4XlXSjPhH6QSGZDzwIQe, garbage
Skipping id: CYsOWtp1ed2XDeplB5O5, garbage
Skipping id: DlXglThZUiPJcuyLLKHV, garbage
Skipping id: NtXdp7ODn4sR3YR2eEW6, garbage
Skipping id: U0DUXOSpfqYsgurOyL3L, garbage
Skipping id: Uikgs6eO85MRivL0peL6, garbage
Skipping id: XIPyRwJloK6ZdAdTHOeu, garbage
Skipping id: d9FxG5haHnN2ku7spAng, garbage
Skipping id: eyzSmD1AdfKkwOYtfoof, garbage
Skipping id: f1ojlkGYgnOI4MLRGC4T, garbage
Skipping id: fySI8xbjfC5hx7QdHpva, garbage
Skipping id: gLDxkmHFL3OFufeje6E8, garbage
Skipping id: jkkOtT3PVoKQMmpA947Y, garbage
Skipping id: l1HjGNlNwpFzTnCsJxJA, garbage
Skipping id: o7jTZJyRoBxaDxIQwyQg, garbage
Skipping id: osfrRcogOlIrEtxp5fpI, garbage
Skipping id: pXXGv1o80FIldPiytsN7, garbage
Skipping id: pf1tiPMceCwvXLUfKQwY, garbage
Skipping id: q2CEZCITA0RVSsfTQRP0, garbage
Skipping id: qAwPXAX5efpoQt0TLoUu, garbage
Skipping id: qXEJ0wGr86cw8CXIgL4z, garbage
Skipping id: r5vxgYs9FnZYOxrh2tcj, garbage
Skipping id: rdguN3u8mWRuiRXfUKM8, garbage
Skipping id

Using 8 omp threads (processes), consider increasing --nb_cores if you have more
Launching the whole pipeline 05/01/2022, 10:25:03
There are 2052 embeddings of dim 768
	Compute estimated construction time of the index 05/01/2022, 10:25:03
		-> Train: 16.7 minutes
		-> Add: 0.0 seconds
		Total: 16.7 minutes
	>>> Finished "Compute estimated construction time of the index" in 0.0001 secs
	Checking that your have enough memory available to create the index 05/01/2022, 10:25:03
7.5MB of memory will be needed to build the index (more might be used if you have more)
	>>> Finished "Checking that your have enough memory available to create the index" in 0.0004 secs
	Selecting most promising index types given data characteristics 05/01/2022, 10:25:03
	>>> Finished "Selecting most promising index types given data characteristics" in 0.0000 secs
	Creating the index 05/01/2022, 10:25:03
		-> Instanciate the index HNSW15 05/01/2022, 10:25:03
		>>> Finished "-> Instanciate the index HNSW15" in 0.0034

100%|██████████| 1/1 [00:00<00:00, 273.01it/s]

		>>> Finished "-> Adding the vectors to the index" in 0.0805 secs
	>>> Finished "Creating the index" in 0.0856 secs
	Computing best hyperparameters 05/01/2022, 10:25:03


	>>> Finished "Computing best hyperparameters" in 2.8843 secs
The best hyperparameters are: efSearch=16383
	Compute fast metrics 05/01/2022, 10:25:06
2000
	>>> Finished "Compute fast metrics" in 6.9442 secs
	Saving the index on local disk 05/01/2022, 10:25:13
	>>> Finished "Saving the index on local disk" in 0.0144 secs
Recap:
{'99p_search_speed_ms': 4.525868840003113,
 'avg_search_speed_ms': 3.4529195985000882,
 'compression ratio': 0.9576115129509153,
 'index_key': 'HNSW15',
 'index_param': 'efSearch=16383',
 'nb vectors': 2052,
 'reconstruction error %': 0.0,
 'size in bytes': 6582778,
 'vectors dimension': 768}
>>> Finished "Launching the whole pipeline" in 10.2528 secs


In [3]:
out_file_name = f"../data/fine_tune_fitness_classification_{datetime.date.today().strftime('%d_%m_%Y')}.jsonl"

for e in memes:
    with open(out_file_name, "a+") as outfile:
        json.dump({
            "prompt": f"{','.join(e['topics'])} ### {e['content']} ~~~",
            "completion": f" 1\n",
        }, outfile)
        outfile.write('\n')
for e in c._firestore_client.collection("deleted_memes").stream():
    # check has "topics" and "content"
    if "topics" not in e.to_dict() or "content" not in e.to_dict():
        continue
    with open(out_file_name, "a+") as outfile:
        json.dump({
            "prompt": f"{','.join(e.get('topics'))} ### {e.get('content')} ~~~",
            "completion": f" 0\n",
        }, outfile)
        outfile.write('\n')

In [7]:
!head -5 $out_file_name | jq .

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{
  "prompt": "ice breaker ### When is a time when you know for sure you'll soon have to exchange a nice, meaningful conversation with someone new? ~~~",
  "completion": " 1\n"
}
{
  "prompt": "space exploration,space travel ### Do you think humans are the only intelligent life in the universe? ~~~",
  "completion": " 1\n"
}
{
  "prompt": "ecology ### Have natural disasters gotten worse with the increase in human existence? If so, why? ~~~",
  "completion": " 1\n"
}
{
  "prompt": "ice breaker ### What would prompt you to make a dramatic change in your life. Is it worth it? ~~~",
  "completion": " 1\n"
}
{
  "prompt": "philosophy,mathematic ### Why bother with subjecti

In [8]:
!echo "execute: openai tools fine_tunes.prepare_data -f $(pwd)/$out_file_name"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
execute: openai tools fine_tunes.prepare_data -f /Users/louisbeaumont/Documents/langame-worker/notebooks/../data/fine_tune_fitness_classification_01_05_2022.jsonl


In [9]:
train_file = openai.File.create(
  file=open("_prepared_train.jsonl"),
  purpose="fine-tune"
)
valid_file = openai.File.create(
  file=open("_prepared_valid.jsonl"),
  purpose="fine-tune"
)
openai.FineTune.create(
    training_file=train_file["id"],
    validation_file=valid_file["id"],
    model="ada",
    # "On classification tasks, we recommend setting this to false"
    use_packing=False,
    # https://beta.openai.com/docs/api-reference/fine-tunes/create#fine-tunes/create-prompt_loss_weight
    # classification recommended 0.1 (it's the default value anyway)
    prompt_loss_weight=0.1
)

<FineTune fine-tune id=ft-bBksgsOcWJjeZuV9XHiHSvIs at 0x158877a90> JSON: {
  "created_at": 1651376064,
  "events": [
    {
      "created_at": 1651376064,
      "level": "info",
      "message": "Created fine-tune: ft-bBksgsOcWJjeZuV9XHiHSvIs",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": null,
    "n_epochs": 4,
    "prompt_loss_weight": 0.1
  },
  "id": "ft-bBksgsOcWJjeZuV9XHiHSvIs",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-KwcHNgfGe4pqdKDLQIJt99UZ",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 586199,
      "created_at": 1651376061,
      "filename": "_prepared_train.jsonl",
      "id": "file-FjpwIA5Q2FH7afulYnA5Z27f",
      "object": "file",
      "purpose": "fine-tune",
      "status": "uploaded",
      "status_details": null
    }
  ],
  "updated_at": 1651376064,
  "validation_files": [
    {
      "bytes": 1

In [10]:
!rm -rf _prepared_train.jsonl _prepared_valid.jsonl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
